In [1]:
import requests
from pyquery import PyQuery as pq
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from datetime import *


location = input("Where ya going?")
startstr = input("What date is the start of your trip?")
endstr = input("When does your trip end?")

def find_date(string):
    if string.count("/") == 1:
        temp = datetime.strptime(string, "%m/%d")
        r = datetime(date.today().year, temp.month, temp.day)
        if r < datetime.now():
            return datetime(r.year + 1, r.month, r.day)
        else:
            return datetime(date.today().year, temp.month, temp.day)
    elif string.count("/") == 2:
        return datetime.strptime(string, "%Y/%m/%d")
    else:
        return (False)
start = find_date(startstr)
end = find_date(endstr)
while (start == False or end == False or end < start or start < datetime.now() or start > datetime.now() + timedelta(days=365) or end > start + timedelta(days=30)):
    print ("Please enter in Month/Day or Year/Month/Day")
    startstr = input("What date is the start of your trip?")
    endstr = input("When does your trip end?")
    start = find_date(startstr)
    end = find_date(endstr)
    

p = input("How many people are going? (Adults Childs)")
people = (int(p.split(" ")[0]), int(p.split(" ")[1]))
child_age = []
for i in range(1, people[1] + 1):
    p = input("How old is child {}?".format(i))
    child_age.append(int(p))
driver = webdriver.Chrome("/Users/Andew1207/pytest/chromedriver")
driver.get("https://www.tripadvisor.com.tw/")
select = driver.find_element_by_css_selector("#component_6 > div > div > div > div")
select.click()
blank = driver.find_element_by_css_selector("#c_redcarpet_1 > div.brand-trip-search-geopill-flyout-GeoPillRedCarpetContents__overlayContents--MBsrM > div > div.brand-trip-search-geopill-flyout-geoPillCommon__container--mKYqH > div.brand-trip-search-geopill-flyout-GeoScopedTypeahead__input_container--1YGE6 > div > input")
blank.send_keys(location)
sleep(0.5)
blank.send_keys(Keys.ENTER)
sleep(0.5)

place_html = pq(requests.get(driver.current_url).text)
hotel_link = "https://www.tripadvisor.com.tw" + place_html("#component_3 > div > div > div > span.ui_column.is-4-mobile.brand-quick-links-QuickLinkTileItem__quickLinkTileItem--zKAkR.is-2-tablet.brand-quick-links-commonStyles__order-hotels--2paOG > div > a").attr("href")
tour_link = "https://www.tripadvisor.com.tw" + place_html("#component_3 > div > div > div > span.ui_column.is-4-mobile.brand-quick-links-QuickLinkTileItem__quickLinkTileItem--zKAkR.is-2-tablet.brand-quick-links-commonStyles__order-attractions--2jENs > div > a").attr("href")
#tour_html = pq(requests.get(tour_link).text)
food_link = "https://www.tripadvisor.com.tw" + place_html("#component_3 > div > div > div > span.ui_column.is-4-mobile.brand-quick-links-QuickLinkTileItem__quickLinkTileItem--zKAkR.is-2-tablet.brand-quick-links-commonStyles__order-restaurants--3elHI > div > a").attr("href")
#food_html = pq(requests.get(food_link).text)
plane_link = "https://www.tripadvisor.com.tw" + place_html("#component_3 > div > div > div > span.ui_column.is-4-mobile.brand-quick-links-QuickLinkTileItem__quickLinkTileItem--zKAkR.is-2-tablet.brand-quick-links-commonStyles__order-flights--3Ww9c > div > a").attr("href")
#plane_html = pq(requests.get(plane_link).text)

driver.get(hotel_link)
sleep(1)
month = None
def clickdate(obj, element1, element2, element3):
    html = pq(driver.find_element_by_css_selector("*").get_attribute("outerHTML"))
    month = html(element1).text()
    for i in range ((obj.month - datetime.strptime(month, "%Y年%m月").month + 12) % 12 ):
        select = driver.find_element_by_css_selector(element2)
        select.click()
        sleep(0.5)
    sleep(0.5)
    weekday = datetime.strftime(datetime(obj.year, obj.month, 1), "%w")
    select = driver.find_element_by_css_selector(element3.format(obj.day + int(weekday)))
    select.click()
sleep(1)
clickdate(start, "span.CHECKIN > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > div > span.rsdc-month-title",
         "span.CHECKIN > div.body_text > div > div.rsdc-body > div.rsdc-next.rsdc-nav.ui_icon.single-chevron-right",
         'span.CHECKIN > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > span > div:nth-child({})')
sleep(1)
clickdate(end, "span.CHECKOUT > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > div >  span.rsdc-month-title",
         "span.CHECKOUT > div.body_text > div > div.rsdc-body > div.rsdc-next.rsdc-nav.ui_icon.single-chevron-right",
         "span.CHECKOUT > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > span > div:nth-child({})")
sleep(0.5)
if people[0] != 2:
    if people[0] < 2:
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.adultsPlaceholder > div > span.rg-ticker > span.rg-ticker-minus.ui_icon.minus")
        for i in range(2 - people[0]):
            select.click()
    else:
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.adultsPlaceholder > div > span.rg-ticker > span.rg-ticker-plus.ui_icon.plus")
        for i in range(people[0] - 2):
            select.click()
sleep(0.5)
select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.childrenPlaceholder > div > span.rg-ticker > span.rg-ticker-plus.ui_icon.plus")
for i in range(people[1]):
    select.click()
for i in range(people[1]):
    try:
        sleep(0.5)
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.ages-wrap > span:nth-child({}) > span > span.picker-inner".format(i + 1))
        select.click()
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > div.ui_overlay.ui_dropdown.prw_rup.prw_ibex_trip_search_rooms_guests_corgi_sft > span > ul.options-container > li:nth-child({})".format(child_age[i] + 1))
        select.click()
        
    except:
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > div.QSISlider.SI_6yB28yrnSys18xf_SliderContainer > div:nth-child(8)")
        select.click()
sleep(0.5)
select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.confirm-text > span")
select.click()

lp = int(input("Lowest price of hotel: "))
hp = int(input("Highest price of hotel: "))
html=pq(requests.get(driver.current_url).text)
Max=html("#component_7 > div > div.hotels-hotels-filters-hotels-filters__container--2BRRK.hotels-hotels-filters-hotels-filters__noTopMargin--2Sy6V > div:nth-child(1) > div.common-filters-FilterWrapper__content--3RxLJ > div > div:nth-child(1) > div.input-slider-ManagedSlider__labels--1xJfl > div:nth-child(2)").text()
a=Max.count(",")
Max=Max.replace(",","",a)
Max=int(Max[1:-1])
while  lp>hp :
    print("out of range")
    lp=int(input("Lowest price of hotel: "))
    hp=int(input("Highest price of hotel: "))
select = driver.find_element_by_css_selector("#component_8 > div.hotels-sort-filter-header-sort-filter-header__headerAndDTSortWrapper--1n9lZ > div.hotels-sort-filter-header-sort-filter-header__DTSortContainer--3OmU8.is-shown-at-desktop > span.hotels-sort-filter-header-sort-dropdown__dropdownWrapper--mgcU6 > div > div")
select.click()
sleep(1)
select = driver.find_element_by_css_selector("#component_8 > div.hotels-sort-filter-header-sort-filter-header__headerAndDTSortWrapper--1n9lZ > div.hotels-sort-filter-header-sort-filter-header__DTSortContainer--3OmU8.is-shown-at-desktop > span.hotels-sort-filter-header-sort-dropdown__dropdownWrapper--mgcU6 > div>div:nth-child(2)>div:nth-child(1)")
select.click()
print (driver.current_url)
sleep(2)
hotel_html = pq(driver.find_element_by_css_selector("*").get_attribute("outerHTML"))
hotels = hotel_html.find("a").filter("[dir='ltr']")
hotels.make_links_absolute(requests.get("https://www.tripadvisor.com.tw/Home-g60763").url)
sleep(1)
a=1
k=1
price= hotel_html("#taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0 > div:nth-child({}) div.premium_offer_container div.price".format(a))
check=price.text().replace(",","",price.text().count(","))
check=check.replace("NT","")
check=check.replace("$","")
for each in hotels.items():
    k+=1
    if check!="":  
        if int(check)<hp and int(check)>lp :
            print (each.text())
            if len(price.text().split(" "))>1:
                print(price.text().split(" ")[1])
                print(each.attr("href"))
            else:    
                print(price.text()) 
                print(each.attr("href"))
    if k%5==0 and k!=30:
        a+=3
    else:
        a+=1
    price= hotel_html("#taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0 > div:nth-child({}) div.premium_offer_container div.price".format(a))
    check=price.text().replace(",","",price.text().count(","))
    check=check.replace("NT","",1)
    check=check.replace("$","",1)
    sleep(0.5)

driver.get(food_link)
sleep(1)
select2=driver.find_element_by_css_selector("#popularity")
select2.click()
food_html = pq(driver.find_element_by_css_selector("*").get_attribute("outerHTML"))
food=food_html.find("a").filter("[class='restaurants-list-ListCell__restaurantName--2aSdo']")
for each in food.items():
    print(each.text())
    sleep(0.1)
    

driver.get(plane_link)
select = driver.find_element_by_css_selector("#roundtrip_picker > div:nth-child(1)")
select.click()
sleep(1)
html = pq(driver.find_element_by_css_selector("*").get_attribute("outerHTML"))
clickdate(datetime(start.year, start.month, start.day) + timedelta(days=2), "span.DEPARTURE > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > span.rsdc-month-title",
         "span.DEPARTURE > div.body_text > div > div.rsdc-body > div.rsdc-next.rsdc-nav.ui_icon.single-chevron-right",
         "span.DEPARTURE > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > span:nth-child({})")
select = driver.find_element_by_css_selector("#roundtrip_picker > div:nth-child(2)")
select.click()
clickdate(datetime(end.year, end.month, end.day) + timedelta(days=2), "span.RETURN > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > span.rsdc-month-title",
         "span.RETURN > div.body_text > div > div.rsdc-body > div.rsdc-next.rsdc-nav.ui_icon.single-chevron-right",
         "span.RETURN > div.body_text > div > div.rsdc-body > div.rsdc-months > span:nth-child(1) > span:nth-child({})")
select = driver.find_element_by_css_selector("#cos_ui_picker > div")
select.click()
select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div > div.travelerRow.adultRow > div.adultCounter.counter > span.ui_icon.plus-circle.enabled")
sleep(1)
for i in range(people[0] - 1):
    select.click()
select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div > div.travelerRow.childrenRow > div.childrenCounter.counter > span.ui_icon.plus-circle.enabled")
sleep(1)
for i in range(people[1]):
    select.click()

for i in range(1, people[1] + 1):
    if i==1:
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div > div.childrenAgesContainer > div.ageContainer.flightsAgeContainer_{} > div.childAgePicker > div ".format(i))
        select.click()
        driver.execute_script("arguments[0].scrollIntoView(true);",select)
        sleep(0.5)
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW >span.ui_overlay.ui_dropdown > div > div:nth-child({})".format(child_age[i - 1]+3))
        select.click()
        
    else:
        select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div > div.childrenAgesContainer > div.ageContainer.flightsAgeContainer_{} > div.childAgePicker > div ".format(i))
        select.click()
        driver.execute_script("arguments[0].scrollIntoView(true);",select)
        sleep(0.5)
        try:
            select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW >span:nth-child({}) > div > div:nth-child({})".format(171+i,child_age[i - 1]+3))
            select.click()
        except:
            try:   
                select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW >span:nth-child({}) > div > div:nth-child({})".format(169+i,child_age[i - 1]+3))
                select.click()
            except:
                try:
                    select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW >span:nth-child({}) > div > div:nth-child({})".format(170+i,child_age[i - 1]+3))
                    select.click()
                except:
                    select = driver.find_element_by_css_selector("#BODY_BLOCK_JQUERY_REFLOW >span:nth-child({}) > div > div:nth-child({})".format(172+i,child_age[i - 1]+3))
                    select.click()

sleep(1)        
select3=driver.find_element_by_css_selector("#search_form_wrapper > div > div.optionsContainer.ui_columns > div > div.providers > div > span > ul > div:nth-child(1) > label")
select3.click()
sleep(1)
select=driver.find_element_by_css_selector("#search_form_wrapper > div > div.optionsContainer.ui_columns > div > div.is-3-tablet.is-3-desktop > button")
select.click()
sleep(3)
select3=driver.find_element_by_css_selector("#c_FL_PopupCFSR > div > div.overlays-pieces-CloseX__close--7erra.overlays-pieces-CloseX__inverted--2vrVh")
select3.click()


plane=pq(driver.find_element_by_css_selector("*").get_attribute("outerHTML"))

recommend = plane("#MAIN > div > div.flights-search-results-SearchContainer__centerColumn--1CrLy.scrollAdMain.withLeftRail > div > div > div > div.flights-search-results-loading-LoadingOverlay__container--3bVRa > div.flights-search-results-FlightList__recommendedList--36U-w > div > div:nth-child(n + 2)")

a=3
l=len (plane("#MAIN > div > div.flights-search-results-SearchContainer__centerColumn--1CrLy.scrollAdMain.withLeftRail > div > div > div > div.flights-search-results-loading-LoadingOverlay__container--3bVRa > div:nth-child(n+3) > div > div > div > div.flights-search-results-itinerary-card-FlightItineraryCard__leftColumn--39i0k >\
    div.flights-search-results-itinerary-card-FlightItineraryCard__itineraryInfo--2Ff-Q > div.flights-search-results-itinerary-card-components-OneWayInfo__container--47EGv > div\
    > div.flights-search-results-itinerary-card-components-OneWayInfo__fareInfoContainer--1wDRe > div.flights-search-results-itinerary-card-components-OneWayInfo__scheduleInfoContainer--piCQR > div.flights-search-results-itinerary-card-components-OneWayInfo__odTime--mp9Dl").text().split("-"))
for k in range(l-1): 
    plane1=plane("#MAIN > div > div.flights-search-results-SearchContainer__centerColumn--1CrLy.scrollAdMain.withLeftRail > div > div > div > div.flights-search-results-loading-LoadingOverlay__container--3bVRa > div:nth-child({}) > div > div > div > div.flights-search-results-itinerary-card-FlightItineraryCard__leftColumn--39i0k >\
     div.flights-search-results-itinerary-card-FlightItineraryCard__itineraryInfo--2Ff-Q > div.flights-search-results-itinerary-card-components-OneWayInfo__container--47EGv > div\
     > div.flights-search-results-itinerary-card-components-OneWayInfo__fareInfoContainer--1wDRe > div.flights-search-results-itinerary-card-components-OneWayInfo__scheduleInfoContainer--piCQR > div.flights-search-results-itinerary-card-components-OneWayInfo__odTime--mp9Dl".format(a))
    plane2=plane("#MAIN > div > div.flights-search-results-SearchContainer__centerColumn--1CrLy.scrollAdMain.withLeftRail > div > div > div > div.flights-search-results-loading-LoadingOverlay__container--3bVRa > div:nth-child({})> div > div > div > div.flights-search-results-itinerary-card-FlightItineraryCard__leftColumn--39i0k > \
     div.flights-search-results-itinerary-card-FlightItineraryCard__itineraryInfo--2Ff-Q > div.flights-search-results-itinerary-card-components-OneWayInfo__lastContainer--1MGqi > div\
     > div.flights-search-results-itinerary-card-components-OneWayInfo__fareInfoContainer--1wDRe > div.flights-search-results-itinerary-card-components-OneWayInfo__scheduleInfoContainer--piCQR > div.flights-search-results-itinerary-card-components-OneWayInfo__odTime--mp9Dl".format(a))
    print("go",plane1.text())
    print("back",plane2.text())
    a+=1
driver.quit()

Where ya going?new york
What date is the start of your trip?10/1
When does your trip end?10/20
How many people are going? (Adults Childs)2 2
How old is child 1?3
How old is child 2?5


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 75
